In [ ]:
!pip install transformers
!pip install datasets
!pip install apache_beam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 11.7 MB/s 
     |████████████████████████████████| 182 kB 63.8 MB/s 
     |████████████████████████████████| 7.6 MB 52.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 27.0 MB/s 
     |████████████████████████████████| 212 kB 69.0 MB/s 
     |████████████████████████████████| 132 kB 78.5 MB/s 
     |████████████████████████████████| 127 kB 75.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.5 MB 28.0 MB/s 
     |████████████████████████████████| 2.7 MB 43.7 MB/s 
     |███████████████

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

In [ ]:
!pip show threading

In [ ]:
dataset = load_dataset("wikipedia", "20220301.de", split="train")

Downloading:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.91G [00:00<?, ?B/s]

Dataset wikipedia downloaded and prepared to /root/.cache/huggingface/datasets/wikipedia/20220301.de/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.


In [ ]:
type(dataset)

__main__.Dataset

In [ ]:
dataset[0]

{'id': '1',
 'url': 'https://de.wikipedia.org/wiki/Alan%20Smithee',
 'title': 'Alan Smithee',
 'text': 'Alan Smithee steht als Pseudonym für einen fiktiven Regisseur, der Filme verantwortet, bei denen der eigentliche Regisseur seinen Namen nicht mit dem Werk in Verbindung gebracht haben möchte. Von 1968 bis 2000 wurde es von der Directors Guild of America (DGA) für solche Situationen empfohlen.\n\nAlternative Schreibweisen sind unter anderem die Ursprungsvariante Allen Smithee sowie Alan Smythee und Adam Smithee. Auch zwei teilweise asiatisch anmutende Schreibweisen Alan Smi Thee und Sumishii Aran gehören – so die Internet Movie Database – dazu.\n\nGeschichte\n\nEntstehung \nDas Pseudonym entstand 1968 infolge der Arbeiten am Western-Film Death of a Gunfighter (deutscher Titel Frank Patch – Deine Stunden sind gezählt). Regisseur Robert Totten und Hauptdarsteller Richard Widmark gerieten in einen Streit, woraufhin Don Siegel als neuer Regisseur eingesetzt wurde.\n\nDer Film trug nach Ab

In [ ]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset):
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we git the 10K mark, save to file
        with open(f'./data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
        if file_count ==20:
          break
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'./data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/2665357 [00:00<?, ?it/s]

In [ ]:
training_corpus = (
    dataset[i : i + 1000]["text"]
    for i in range(0, round(len(dataset)/100), 1000)
)

In [ ]:
from pathlib import Path
paths = [str(x) for x in Path('./data').glob('**/*.txt')]

In [ ]:
paths[0]

'data/text_228.txt'

In [ ]:
from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased") 

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/240k [00:00<?, ?B/s]

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [ ]:
import os

#os.mkdir('./trained-tokenizer')

tokenizer.save_pretrained('trained-tokenizer')

('trained-tokenizer/tokenizer_config.json',
 'trained-tokenizer/special_tokens_map.json',
 'trained-tokenizer/vocab.txt',
 'trained-tokenizer/added_tokens.json',
 'trained-tokenizer/tokenizer.json')

In [ ]:
# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = AutoTokenizer.from_pretrained('trained-tokenizer', max_len=512)

In [ ]:
tokens = tokenizer("Ich habe eine Kartoffel")

In [ ]:
print(tokens)

{'input_ids': [2, 9963, 7598, 5436, 27328, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}


In [ ]:
tokens.input_ids

[2, 9963, 7598, 5436, 27328, 3]

In [ ]:
with open('./data/text_0.txt', 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')

In [ ]:
batch = tokenizer(lines, max_length=512, padding='max_length', truncation=True)
len(batch)

3

In [ ]:
ids_list = []
for i in range(0,9999):
  ids_list.append(batch[i].ids)

In [ ]:
mask_list = []
for i in range(0,9999):
  mask_list.append(batch[i].attention_mask)

In [ ]:
import torch

labels = torch.tensor(ids_list)
mask = torch.tensor([mask_list])

In [ ]:
labels[0]

tensor([    2, 16985, 12659, 10088,  6945,  5410, 18167,  5424,  5613, 29872,
         9357,    16,  5318, 10782, 11823,  5343,    16,  5456,  6615,  5318,
        15167,  9357,  6307,  6727,  5517,  5382,  5411,  6223,  5340,  8231,
        10483,  6052, 20901,    18,  6581,  8284,  5526,  6869,  5460,  5551,
         5347,  5318, 46593,  3655, 40405,  5908, 19054,    12, 23583,  3679,
           13,  5424,  9045, 23381, 22568,    18, 16456, 35088,  5525,  5513,
         6848,  5330, 17457, 27762, 16795, 12659, 10088,  5790, 16985, 12471,
        50445,  3646,  5326,  9442, 12659, 10088,    18,  6335,  5717,  7562,
        11913,  8734,  5332, 47845,  7596,  5365, 35088, 16985, 12471,  3654,
         5707,  3646,  5326,  9649, 38029,  3654, 15159,  3658,  7792,  1687,
         5470,  5330,  8912, 36864, 38973,  1687,  7077,    18,  6199,  3713,
         5388, 43226,  5584, 18167,  6570,  8284, 12509,  5318, 10117,  5479,
        18663,    17,  6157, 24038,  5908,    69, 23221, 41111, 

In [ ]:
mask[0][50]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
len(mask_list)

9999

In [ ]:
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3

In [ ]:
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()

In [ ]:
input_ids

tensor([[    2, 16985, 12659,  ..., 19395, 23022,     3],
        [    2, 12494,  8508,  ..., 10439,  6099,     3],
        [    2,  6127, 14284,  ...,  5318, 10607,     3],
        ...,
        [    2, 22706,    12,  ...,  5528,  1697,     3],
        [    2,  6973,  6945,  ...,     0,     0,     0],
        [    2, 29212,  5523,  ...,  6861,  5318,     3]])

In [ ]:
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)

In [ ]:
rand

tensor([[0.3408, 0.0589, 0.0852,  ..., 0.5547, 0.1606, 0.6814],
        [0.5194, 0.0841, 0.5483,  ..., 0.2398, 0.1039, 0.0725],
        [0.3924, 0.8853, 0.8651,  ..., 0.6719, 0.8154, 0.7119],
        ...,
        [0.4241, 0.1863, 0.2204,  ..., 0.7932, 0.3800, 0.1801],
        [0.3637, 0.3526, 0.9398,  ..., 0.7138, 0.8144, 0.6737],
        [0.5338, 0.2734, 0.1168,  ..., 0.3229, 0.8224, 0.5408]])

In [ ]:
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)

In [ ]:
mask_arr

tensor([[False,  True,  True,  ..., False, False, False],
        [False,  True, False,  ..., False,  True,  True],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False]])

In [ ]:
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3

In [ ]:
input_ids[0][:200]

tensor([    2,     3,     3, 10088,  6945,  5410, 18167,     3,  5613, 29872,
         9357,    16,     3, 10782, 11823,  5343,    16,  5456,     3,  5318,
        15167,  9357,     3,  6727,  5517,  5382,  5411,  6223,  5340,  8231,
        10483,  6052,     3,    18,  6581,  8284,  5526,  6869,  5460,  5551,
         5347,  5318, 46593,     3, 40405,  5908, 19054,    12,     3,  3679,
           13,  5424,  9045, 23381, 22568,    18,     3, 35088,     3,  5513,
         6848,  5330, 17457, 27762,     3, 12659, 10088,  5790, 16985,     3,
        50445,  3646,  5326,  9442, 12659, 10088,    18,  6335,  5717,  7562,
        11913,  8734,  5332, 47845,  7596,  5365, 35088, 16985, 12471,  3654,
         5707,  3646,  5326,  9649, 38029,  3654, 15159,  3658,  7792,  1687,
         5470,  5330,  8912, 36864, 38973,  1687,  7077,    18,  6199,  3713,
         5388, 43226,  5584, 18167,  6570,     3, 12509,  5318, 10117,  5479,
            3,    17,  6157, 24038,  5908,    69, 23221, 41111, 

In [ ]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        diction = {}
        diction['input_ids'] = encodings['input_ids'][i]
        diction['attention_mask'] = encodings['attention_mask'][0][i]
        diction['labels'] = encodings['labels'][i]
        return diction

In [ ]:
dataset = Dataset(encodings)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52000,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/1250 [00:00<?, ?it/s]

  0%|          | 0/1250 [00:00<?, ?it/s]

In [ ]:
model.save_pretrained('./test-pretrain')

In [ ]:
from transformers import pipeline

In [ ]:
fill = pipeline('fill-mask', model='test-pretrain', tokenizer='trained-tokenizer')

In [ ]:
fill(f'Alan Smithee steht {fill.tokenizer.mask_token} Pseudonym')

[{'score': 0.21680769324302673,
  'token': 4,
  'token_str': '[MASK]',
  'sequence': 'Alan Smithee steht Pseudonym'},
 {'score': 0.0002167739876313135,
  'token': 39922,
  'token_str': 'Kirchenlieddichter',
  'sequence': 'Alan Smithee steht Kirchenlieddichter Pseudonym'},
 {'score': 0.00020904833218082786,
  'token': 31249,
  'token_str': '##miner',
  'sequence': 'Alan Smithee stehtminer Pseudonym'},
 {'score': 0.00018966064089909196,
  'token': 19547,
  'token_str': 'SF',
  'sequence': 'Alan Smithee steht SF Pseudonym'},
 {'score': 0.00018130695389118046,
  'token': 45701,
  'token_str': '##asens',
  'sequence': 'Alan Smithee stehtasens Pseudonym'}]

In [ ]:
lines[0]

'Alan Smithee steht als Pseudonym für einen fiktiven Regisseur, der Filme verantwortet, bei denen der eigentliche Regisseur seinen Namen nicht mit dem Werk in Verbindung gebracht haben möchte. Von 1968 bis 2000 wurde es von der Directors Guild of America (DGA) für solche Situationen empfohlen.Alternative Schreibweisen sind unter anderem die Ursprungsvariante Allen Smithee sowie Alan Smythee und Adam Smithee. Auch zwei teilweise asiatisch anmutende Schreibweisen Alan Smi Thee und Sumishii Aran gehören – so die Internet Movie Database – dazu.GeschichteEntstehung Das Pseudonym entstand 1968 infolge der Arbeiten am Western-Film Death of a Gunfighter (deutscher Titel Frank Patch – Deine Stunden sind gezählt). Regisseur Robert Totten und Hauptdarsteller Richard Widmark gerieten in einen Streit, woraufhin Don Siegel als neuer Regisseur eingesetzt wurde.Der Film trug nach Abschluss der Arbeiten noch deutlich Tottens Handschrift, der auch mehr Drehtage als Siegel daran gearbeitet hatte, weshalb